# User Steps for Real-World Data

### Team Epsilon-Greedy Quants
#### Michael Lee, Nikat Patel, Jose Antonio Alatorre Sanchez

This is a user guide for how to load real-world data into our model and how to perform Policy-Gradient Methods such as REINFORCE, REINFORCE with Baseline, Actor-Critic, and Actor-Critic with Eligibility Traces.  The completed versions of this user guide showing what the model results look like after the code is executed is shown the **Real-World Data Demonstration** directory.

In [ ]:
from environments.e_greedy import DeepTradingEnvironment, LinearAgent

import datetime
import numpy as np
import pandas as pd
import os
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.cla import CLA
import matplotlib.pyplot as plt
from matplotlib import cm
import copy
import quantstats as qs
qs.extend_pandas()

In [ ]:
root = os.getcwd()
data_env = root+"/data_env/"

# Utility Functions

In [ ]:
def _retrieve_asset_dict():
    # obtain close prices from parquet files of ETF price history
    root = os.getcwd()
    data_env = root+"/data_env/"
    files = [_ for _ in os.listdir(data_env) if "parquet" in _]
    assets_dict = {file: pd.read_parquet(data_env + "/" + file) for file in files}
    counter=0
    for key, value in assets_dict.items():
        if counter==0:
            main_index=value.index
    else:
        main_index=main_index.join(value.index,how="inner")
        
    for key, value in assets_dict.items():
        tmp_df=value.reindex(main_index)
        tmp_df=tmp_df.fillna(method='ffill')
        assets_dict[key]=tmp_df['close']
    return assets_dict

def build_portfolio_df(asset_dict):
    portfolio_df = pd.DataFrame()
    
    for key, value in assets_dict.items():
        key = key.split(".")[0]
        tmp_df = pd.DataFrame(data=value)
        tmp_df.columns=[key]
        portfolio_df = pd.concat([portfolio_df, tmp_df], axis=1)
        
    portfolio_df.index = pd.to_datetime(portfolio_df.index, errors='coerce')
    return portfolio_df

In [ ]:
def plot_backtest(linear_agent_train, env_test, test_input, model_run, model):
    ## Create plot of backtest returns
    if not "backtest" in locals():
        backtest=None
    
    benchmark_max_return, benchmark_max_sharpe, benchmark_min_volatility = \
        linear_agent_train._run_benchmarks(
            test_input, 
            df_start=portfolio_df_test.iloc[0].name, 
            df_end=portfolio_df_test.iloc[-1].name, 
            benchmark_start=portfolio_df_test.iloc[14].name)

    test_input = test_input[14:]
    backtest, tmp_weights =linear_agent_train.backtest_policy(epoch=1,backtest=backtest, env_test=env_test, test_input=test_input)
    backtest = backtest[portfolio_df_test.iloc[14].name:]
    plt.figure(figsize=(12, 6))
    
    backtest_return = round((backtest[1][-1] - 1) * 100, 2)
    backtest_volatility = round(backtest[1].std() * 100, 2)
    plt.plot(backtest, color="blue", label="Test Data - Return: {}%, Volatility: {}%".format(str(backtest_return), str(backtest_volatility)))
    
    # plot benchmarks
    plt.plot(benchmark_max_return, color="black", label="Real Dataset Benchmark - Max Return, Return: {}%, Volatility: {}%".format(round((benchmark_max_return[-1] - 1) * 100, 2), round(benchmark_max_return.std() * 100, 2)))    
    plt.plot(benchmark_max_sharpe, color="orange", label="Real Dataset Benchmark - Max Sharpe, Return: {}%, Volatility: {}%".format(round((benchmark_max_sharpe[-1] - 1) * 100, 2), round(benchmark_max_sharpe.std() * 100, 2))) 
    plt.plot(benchmark_min_volatility, color="green",  label="Real Dataset Benchmark - Min Volatility, Return: {}%, Volatility: {}%".format( round((benchmark_min_volatility[-1] - 1) * 100, 2), round(benchmark_min_volatility.std() * 100, 2)))

    plt.gcf().autofmt_xdate()
    plt.xticks(fontsize=linear_agent_train.tick_size)
    plt.yticks(fontsize=linear_agent_train.tick_size)
    plt.xlabel("Date", fontsize = linear_agent_train.label_size)
    plt.ylabel("Backtest", fontsize = linear_agent_train.label_size)
    plt.title("Backtest on Test Data: "+ model,fontsize = linear_agent_train.title_size)
    plt.legend(loc="upper left")
    plt.savefig("{}/{}_test_backtest_plot_{}.png".format(linear_agent_train.model_run_dir, model_run, model))
    tmp_weights.to_csv(root+'/temp_persisted_data/'+model_run+"_test_backtest_weights_"+model+'.csv')
    plt.show()
    return backtest

# Reviewing Real-World Data

The real-world data that we use for our models is located in the "data_env" folder.  If you have other assets you would like to use, please obtain a time series of daily time bars and save it in the "data_env" folder as a separate parquet file for each asset.  While all asset price quote history can be included (open, high, low, close, volume) in the parquet file, please ensure that the closing price history is included under a column named "close" as we will be using the closing price for our model inputs.  Below is an example of what our ETF data history files contain.   

In [ ]:
# read a sample ETF
pd.read_parquet(data_env+'EEMV.parquet').head()

To view what a portfolio of ETFs looks like, we create a portolio with the ETF files which are in the "data_env" folder.  We can then split these datasets into train and test sets for testing our model performances.

In [ ]:
# create a portfolio
assets_dict = _retrieve_asset_dict()
portfolio_df = build_portfolio_df(assets_dict)

In [ ]:
# create a train dataset and de-mean the time series

portfolio_df_train = portfolio_df[portfolio_df.index >= '2019-02-01']
portfolio_df_train = portfolio_df_train[portfolio_df_train.index <= '2020-02-01']
portfolio_df_train.sub(portfolio_df_train.mean())

portfolio_df_train.head()

In [ ]:
# create a test dataset consisting of 6 months of data and de-mean the time series

portfolio_df_test = portfolio_df[portfolio_df.index >= '2020-03-15']
portfolio_df_test = portfolio_df_test[portfolio_df_test.index <= '2020-11-16']
portfolio_df_test.sub(portfolio_df_test.mean())

portfolio_df_test.head()

# Set Up Environment

Here, we set up the environment to load ETFs from the "data_env" folder and persist data transformations for further training.  We also specify the meta parameters and objective parameters we want for the environment.

In [ ]:
# parameters related to the transformation of data, this parameters govern an step before the algorithm
out_reward_window=datetime.timedelta(days=7)

meta_parameters = {"in_bars_count": 1,
                   "out_reward_window":out_reward_window ,
                   "state_type":"in_window_out_window",
                   "risk_aversion":0,
                   "include_previous_weights":False}

# parameters that are related to the objective/reward function construction
objective_parameters = {"percent_commission": .001}

print("===Meta Parameters===")
print(meta_parameters)
print("===Objective Parameters===")
print(objective_parameters)

# create an environment and build features based on Real-World Dataset located in the "data_env" folder 
env = DeepTradingEnvironment.build_environment_from_dirs_and_transform(meta_parameters, objective_parameters,data_hash="real_data", data_dir="data_env")

number_of_assets = env.number_of_assets

#### Split Features and Forward Returns into Training and Test sets

For performing backtest return analysis, we split the features and forward returns that were generated by the "build_environment_from_dirs_and_transform" function into a training set and a test set.  These features and forward returns are then de-meaned and will be used to create two separate environments for training and test.

In [ ]:
features = pd.read_parquet("temp_persisted_data/only_features_real_data")

features_train = features[features.index >= '2019-02-01']
features_train = features_train[features_train.index <= '2020-02-01']

features_test = features[features.index >= '2020-03-15']
features_test = features_test[features_test.index <= '2020-11-16']

features_train.head()

In [ ]:
forward_return_dates = pd.read_parquet("temp_persisted_data/forward_return_dates_real_data")

forward_return_dates_train = forward_return_dates[forward_return_dates.index >= '2019-02-01']
forward_return_dates_train = forward_return_dates_train[forward_return_dates_train.index <= '2020-02-01']

forward_return_dates_test = forward_return_dates[forward_return_dates.index > '2020-03-15']
forward_return_dates_test = forward_return_dates_test[forward_return_dates_test.index <= '2020-11-16']

forward_return_dates_train.head()

In [ ]:
forward_returns = pd.read_parquet("temp_persisted_data/only_forward_returns_real_data")

forward_returns_train = forward_returns[forward_returns.index >= '2019-02-01']
forward_returns_train = forward_returns_train[forward_returns_train.index <= '2020-02-01']
forward_returns_train.sub(forward_returns_train.mean())

forward_returns_test = forward_returns[forward_returns.index >= '2020-03-15']
forward_returns_test = forward_returns_test[forward_returns_test.index <= '2020-11-16']
forward_returns_test.sub(forward_returns_test.mean())

forward_returns_train.head()

# Run Policy Gradient Method Algorithms on Real-World Data

Now that we have our features and forward returns divded into training and test set, we can perform our policy gradient method algorithms on the real-world data which we loaded.  

- For each PGM, we first create a train and test environment based on the features and forward_returns of that data.  

- We then instantiate a Linear Agent based on the train environment and a specified reward function (we are currently using "return_with_variance_risk" but one could also use "cum_return", "max_sharpe", "min_vol", or "min_variance").

- Once the Linear Agent is instantiated, we then call REINFORCE_fit() or ACTOR_CRITIC_fit().  We can choose whether to include baseline for REINFORCE or eligibility traces for Actor-Critic by specifying the appropriate flag when calling the function).

- As the model runs, the progress status is listed and status plots are displayed based on the interval of plotting the user specified.  To turn off the print display, set verbose=False when calling the model function.

- Once the model has completed training, view the backtest returns of the test dataset by calling the plot_backtest() function.  Backtest results can be saved to a CSV file.

In [ ]:
max_iter = 4001
model_run = "Full_Portfolio_lambda_0"
sample_observations = 4

### REINFORCE

In [ ]:
# create environment and run REINFORCE

env_reinforce_train=DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_reinforce_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_reinforce = LinearAgent(environment=env_reinforce_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_reinforce._load_benchmark(portfolio_df)
linear_agent_reinforce.REINFORCE_fit(max_iterations=max_iter, add_baseline=False, verbose=True)

In [ ]:
# perform backtest
backtest_reinforce = plot_backtest(linear_agent_reinforce, env_reinforce_test, portfolio_df_test, model="REINFORCE")
backtest_reinforce.to_csv('temp_persisted_data/'+model_run+'backtest_reinforce.csv')

### REINFORCE with Baseline

In [ ]:
# create environment and run REINFORCE with baseline
env_reinforce_baseline_train = DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_reinforce_baseline_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_reinforce_baseline = LinearAgent(environment=env_reinforce_baseline_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_reinforce_baseline._load_benchmark(portfolio_df)
linear_agent_reinforce_baseline.REINFORCE_fit(max_iterations=max_iter, add_baseline=True, verbose=True)

In [ ]:
# perform backtest
backtest_reinforce_baseline = plot_backtest(linear_agent_reinforce_baseline, env_reinforce_baseline_test, portfolio_df_test, model="REINFORCE with Baseline")
backtest_reinforce_baseline.to_csv('temp_persisted_data/'+model_run+'backtest_reinforce_baseline.csv')

### Actor-Critic

In [ ]:
# create environment and run Actor-Critic 

env_actor_critic_no_trace_train = DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_actor_critic_no_trace_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_actor_critic_no_trace = LinearAgent(environment=env_actor_critic_no_trace_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_actor_critic_no_trace._load_benchmark(portfolio_df)
linear_agent_actor_critic_no_trace.ACTOR_CRITIC_FIT(use_traces=False,max_iterations=max_iter, verbose=True)

In [ ]:
# perform backtest 
backtest_actor_critic_no_trace = plot_backtest(linear_agent_actor_critic_no_trace, env_actor_critic_no_trace_test,  portfolio_df_test, model="Actor-Critic without Eligibility Traces")
backtest_actor_critic_no_trace.to_csv('temp_persisted_data/'+model_run+'backtest_actor_critic_no_trace.csv')

### Actor-Critic with Eligibility Traces

In [ ]:
# create environment and run Actor-Critic with Eligibility Traces 
env_actor_critic_trace_train = DeepTradingEnvironment(features_train, forward_returns_train, forward_return_dates_train, objective_parameters,
                 meta_parameters)
env_actor_critic_trace_test = DeepTradingEnvironment(features_test, forward_returns_test, forward_return_dates_test, objective_parameters,
                 meta_parameters)

linear_agent_actor_critic_trace = LinearAgent(environment=env_actor_critic_trace_train,out_reward_window_td=out_reward_window, reward_function="return_with_variance_risk",sample_observations=sample_observations)
linear_agent_actor_critic_trace.ACTOR_CRITIC_FIT(use_traces=True,max_iterations=max_iter, verbose=True)

In [ ]:
# perform backtest
backtest_actor_critic_trace = plot_backtest(linear_agent_actor_critic_trace, env_actor_critic_trace_test,  portfolio_df_test, model="Actor-Critic with Eligibility Traces")
linear_agent_actor_critic_trace._load_benchmark(portfolio_df)
backtest_actor_critic_trace.to_csv('temp_persisted_data/'+model_run+'backtest_actor_critic_trace.csv')